# Script de treinamento com SDK do Python

In [28]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

credential = DefaultAzureCredential()

ml_client = MLClient(
    credential,
    subscription_id="Azure subscription 1",
    resource_group="rg-dio-dp-100",
    workspace_name="workspace-dio-dp-100" 
)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [30]:
ml_client = MLClient.from_config(credential=credential)

Found the config file in: .\config.json


Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [ ]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#load the dataset
print("Loading the dataset...")
sorvetes = pd.read_csv('src/vendas_sorvetes.csv')


#separate features and labels
X, y = sorvetes[['Temperatura (°C)']].values, sorvetes[[Quantidade de Sorvetes]].values

#split the dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#set regularization hyperparameter
reg=0.01

#train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver='liblinear').fit(X_train, y_train)

#calculate accuracy
y_hat = model
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

#caculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_scores[:, 1])
print('AUC:'+ str(auc)) 

#src/sorvetes-training.py

In [ ]:
from azure.ai.ml import command

#configure job
job = command(
    code ="./src",
    command = "python sorvetes-training.py",
    enviroment = "AzureML-sklean-0.24-ubuntu20.04-py38-cpu:1",
    compute = "aml-cluster",
    display_name = "Training a logistic regression model",
    experiment_name = "sorvetes-training",
)

#submit job
returned_job = ml_client.jobs.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)
